In [7]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import pickle

books=pd.read_csv('books.csv')
users=pd.read_csv('users.csv')
ratings=pd.read_csv('ratings.csv')
print(books.shape)
print(users.shape)
print(ratings.shape)

#Popularity Based Recommender System

ratings_with_name = ratings.merge(books,on='ISBN')
num_rating_df = ratings_with_name.groupby('Book-Title').count()['Book-Rating'].reset_index()
num_rating_df.rename(columns={'Book-Rating':'num_ratings'},inplace=True)
num_rating_df

avg_rating_df = ratings_with_name.groupby('Book-Title').mean()['Book-Rating'].reset_index()
avg_rating_df.rename(columns={'Book-Rating':'avg_ratings'},inplace=True)
avg_rating_df

popular_df=num_rating_df.merge(avg_rating_df,on='Book-Title')

popular_df = popular_df[popular_df['num_ratings']>=250].sort_values('avg_ratings',ascending=False).head(50)
popular_df = popular_df.merge(books,on='Book-Title').drop_duplicates('Book-Title')[['Book-Title','Book-Author','Image-URL-M','num_ratings','avg_ratings']]
popular_df['Image-URL-M']

#Collaborative Filtering based Recommender System
x = ratings_with_name.groupby('User-ID').count()['Book-Rating'] > 200
experienced_users= x[x].index
filtered_rating = ratings_with_name[ratings_with_name['User-ID'].isin(experienced_users)]
y = filtered_rating.groupby('Book-Title').count()['Book-Rating']>=50
famous_books=y[y].index
final_ratings = filtered_rating[filtered_rating['Book-Title'].isin(famous_books)]
pt = final_ratings.pivot_table(index='Book-Title', columns='User-ID',values='Book-Rating')
pt.fillna(0,inplace=True)
similarity_scores = cosine_similarity(pt)
similarity_scores.shape

def recommend(book_name):
    #index fetch[0]
    index = np.where(pt.index=='Zoya')[0][0]
    similar_items = sorted(list(enumerate(similarity_scores[0])),key=lambda x:x[1], reverse=True)[1:6]
    data=[]
    for i in similar_items:
        item=[]
        temp_df = books[books['Book-Title'] == pt.index[i[0]]]
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Title'].values))
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Author'].values))
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Image-URL-M'].values))
        data.append(item)
    return data

recommend('1984')

pickle.dump(popular_df,open('popular.pkl','wb'))

pickle.dump(pt,open('pt.pkl','wb'))
pickle.dump(books,open('books.pkl','wb'))
pickle.dump(similarity_scores,open('similarity_scores.pkl','wb'))


popular_df['Image-URL-M'][0]

c:\users\sharmi dev gupta\appdata\local\programs\python\python39\lib\site-packages\IPython\core\interactiveshell.py:3169: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(271360, 8)
(278858, 3)
(1149780, 3)


'http://images.amazon.com/images/P/0439136350.01.MZZZZZZZ.jpg'